In [137]:
import xarray as xr
import numpy as np
from glob import glob
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=15, threads_per_worker=1)
client  = Client(cluster, asynchronous=True)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 15,Total memory: 125.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:39969,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 15
Started: Just now,Total memory: 125.57 GiB
Comm: tcp://127.0.0.1:45973,Total threads: 1
Dashboard: http://127.0.0.1:35977/status,Memory: 8.37 GiB
Nanny: tcp://127.0.0.1:43583,


In [138]:
masks = xr.open_dataset('data/MASKS.nc')

In [152]:
dirpaths = sorted(glob('data/S2S/reforecasts/*'))[:-40]
hovmoller_tropical   = []
hovmoller_coastnorth = []
hovmoller_coastsouth = []
for directory in dirpaths:
    print(directory)
    paths = glob(directory+'/*_SSH.nc')
    data1  = []
    for p in paths:
        d = xr.open_dataset(p).squeeze()
        d = d.sortby('lat').sortby('lon').sel({'lon':slice(220,290),'lat':slice(-40,40)})
        d = d.rename({'time':'leadtime'})
        d.coords['inittime'] = d.leadtime[0].values
        d['leadtime'] = np.arange(len(d.leadtime))
        data1.append(d['dslm'])
    data1     = xr.concat(data1, 'inittime').drop('depth').sortby('inittime').sortby('leadtime')
    
    paths = glob(directory+'/*_300T.nc')
    data2  = []
    for p in paths:
        d = xr.open_dataset(p).squeeze()
        d = d.sortby('lat').sortby('lon').sel({'lon':slice(220,290),'lat':slice(-40,40)})
        d = d.rename({'time':'leadtime'})
        d.coords['inittime'] = d.leadtime[0].values
        d['leadtime'] = np.arange(len(d.leadtime))
        data2.append(d['param18.4.10'])
    data2     = xr.concat(data2, 'inittime').drop('depth').sortby('inittime').sortby('leadtime')
    data = xr.merge([data1,data2])
    
    tropical = data.where(masks.tropicalmask).mean('lat').dropna('lon').sortby('lon')
    tropical.coords['index'] = ('lon',np.arange(len(tropical.lon)))
    # tropical = tropical.to_dataset(name=var[1]+'_tropical')
    
    coast_north = data.where(masks.coastmask_north).mean('lon').dropna('lat').sortby('lat')
    coast_north.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_north.lat)))
    # coast_north = coast_north.to_dataset(name=var[1]+'_coastnorth')
    
    coast_south = data.where(masks.coastmask_south).mean('lon').dropna('lat').sortby('lat', ascending=False)
    coast_south.coords['index'] = ('lat', 1+tropical.index[-1].item()+np.arange(len(coast_south.lat)))
    # coast_south = coast_south.to_dataset(name=var[1]+'_coastsouth')
    
    hovmoller_tropical.append(tropical.rename({'dslm':'ssh_tropical', 'param18.4.10':'T300_tropical'}))
    hovmoller_coastnorth.append(coast_north.rename({'dslm':'ssh_coastnorth', 'param18.4.10':'T300_coastnorth'}))
    hovmoller_coastsouth.append(coast_south.rename({'dslm':'ssh_coastsouth', 'param18.4.10':'T300_coastsouth'}))
    
print('joining')
hovmoller_tropical   = xr.concat(hovmoller_tropical, 'inittime').sortby('inittime')
hovmoller_coastnorth = xr.concat(hovmoller_coastnorth, 'inittime').sortby('inittime')
hovmoller_coastsouth = xr.concat(hovmoller_coastsouth, 'inittime').sortby('inittime')

data/S2S/reforecasts/2020-01-06
data/S2S/reforecasts/2020-01-09
data/S2S/reforecasts/2020-01-13
data/S2S/reforecasts/2020-01-16
data/S2S/reforecasts/2020-01-20
data/S2S/reforecasts/2020-01-23
data/S2S/reforecasts/2020-01-27
data/S2S/reforecasts/2020-01-30
data/S2S/reforecasts/2020-02-03
data/S2S/reforecasts/2020-02-06
data/S2S/reforecasts/2020-02-10
data/S2S/reforecasts/2020-02-13
data/S2S/reforecasts/2020-02-17
data/S2S/reforecasts/2020-02-20
data/S2S/reforecasts/2020-02-24
data/S2S/reforecasts/2020-02-27
data/S2S/reforecasts/2020-03-02
data/S2S/reforecasts/2020-03-05
data/S2S/reforecasts/2020-03-09
data/S2S/reforecasts/2020-03-12
data/S2S/reforecasts/2020-03-16
data/S2S/reforecasts/2020-03-19
data/S2S/reforecasts/2020-03-23
data/S2S/reforecasts/2020-03-26
data/S2S/reforecasts/2020-03-30
data/S2S/reforecasts/2020-04-02
data/S2S/reforecasts/2020-04-06
data/S2S/reforecasts/2020-04-09
data/S2S/reforecasts/2020-04-13
data/S2S/reforecasts/2020-04-16
data/S2S/reforecasts/2020-04-20
data/S2S

In [156]:
hovmoller_tropical.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_tropical.nc')
hovmoller_coastnorth.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_coastnorth.nc')
hovmoller_coastsouth.to_netcdf('data/S2S/HOVMOLLERS/reforecasts_coastsouth.nc')

In [158]:
client.shutdown()